In [2]:
import pyarrow # must occur prior to ray import
import ray
from ray import tune
from ray.tune import ExperimentAnalysis
from ray.tune.search.hyperopt import HyperOptSearch
import datetime
import numpy as np
import pandas as pd
import random
import seaborn as sns; sns.set()
from collections import Counter
from datasets import load_from_disk
from scipy.stats import ranksums
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
from geneformer import TranscriptomeTokenizer
import anndata as ad
import pandas as pd

2023-11-17 13:10:08,397	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-17 13:10:08,842	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
train_dataset=load_from_disk("/root/autodl-tmp/AF_T_cell.dataset")

In [1]:
num_proc=30
train_dataset

NameError: name 'train_dataset' is not defined

In [4]:
conditions = train_dataset["condition"]
target_names = list(set(conditions))

In [5]:
target_names

['Ctrl', 'AF']

In [6]:
target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))

In [7]:
target_name_id_dict

{'Ctrl': 0, 'AF': 1}

In [8]:
trainset_v3 = train_dataset.rename_column("condition","label")

In [9]:
trainset_v3

Dataset({
    features: ['input_ids', 'cell_type', 'label', 'length'],
    num_rows: 20152
})

In [10]:
def classes_to_ids(example):
    example["label"] = target_name_id_dict[example["label"]]
    return example

trainset_v4 = trainset_v3.map(classes_to_ids, num_proc=num_proc)

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [11]:
indiv_list = trainset_v4["input_ids"]

In [12]:
len(indiv_list)

20152

In [13]:
random.seed(42)
train_indiv = random.sample(indiv_list,round(0.7*len(indiv_list)))
eval_indiv = [indiv for indiv in indiv_list if indiv not in train_indiv]
valid_indiv = random.sample(eval_indiv,round(0.5*len(eval_indiv)))
test_indiv = [indiv for indiv in eval_indiv if indiv not in valid_indiv]

In [14]:
def if_train(example):
    return example["input_ids"] in train_indiv

classifier_trainset = trainset_v4.filter(if_train,num_proc=num_proc).shuffle(seed=42)

In [15]:
def if_valid(example):
    return example["input_ids"] in valid_indiv

classifier_validset = trainset_v4.filter(if_valid,num_proc=num_proc).shuffle(seed=42)

In [16]:
def if_test(example):
    return example["input_ids"] in test_indiv

classifier_testdset = trainset_v4.filter(if_test,num_proc=num_proc).shuffle(seed=42)

In [17]:
current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
output_dir = f"/root/autodl-tmp/T_cell/model/{datestamp}_geneformer_DiseaseClassifier/"

In [18]:
import os
saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
if os.path.isfile(saved_model_test) == True:
    raise Exception("Model already saved to this directory.")

In [19]:
# make output directory
import subprocess
subprocess.call(f'mkdir {output_dir}', shell=True)

mkdir: cannot create directory ‘/root/autodl-tmp/T_cell/model/231116_geneformer_DiseaseClassifier/’: File exists


1

In [66]:
# set training parameters
# how many pretrained layers to freeze
freeze_layers = 2
# batch size for training and eval
geneformer_batch_size = 12
# number of epochs
epochs = 50
# logging steps
logging_steps = round(len(classifier_trainset)/geneformer_batch_size/10)

In [67]:
def model_init():
    model = BertForSequenceClassification.from_pretrained("/root/autodl-tmp/Geneformer/",
                                                          num_labels=len(target_names),
                                                          output_attentions = False,
                                                          output_hidden_states = False)
    if freeze_layers is not None:
        modules_to_freeze = model.bert.encoder.layer[:freeze_layers]
        for module in modules_to_freeze:
            for param in module.parameters():
                param.requires_grad = False

    return model

In [68]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy and macro f1 using sklearn's function
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_f1': macro_f1
    }

In [69]:
training_args = {
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "steps",
    "eval_steps": logging_steps,
    "logging_steps": logging_steps,
    "group_by_length": True,
    "length_column_name": "length",
    "disable_tqdm": True,
    "skip_memory_metrics": True, # memory tracker causes errors in raytune
    "per_device_train_batch_size": geneformer_batch_size,
    "per_device_eval_batch_size": geneformer_batch_size,
    "num_train_epochs": epochs,
    "load_best_model_at_end": True,
    "output_dir": output_dir,
    "save_steps":118,
}

In [70]:
training_args_init = TrainingArguments(**training_args)

In [71]:
trainer = Trainer(
    model_init=model_init,
    args=training_args_init,
    data_collator=DataCollatorForCellClassification(),
    train_dataset=classifier_trainset,
    eval_dataset=classifier_validset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
ray_config = {
    "num_train_epochs": tune.choice([epochs]),
    "learning_rate": tune.loguniform(1e-6, 1e-3),
    "weight_decay": tune.uniform(0.0, 0.3),
    "lr_scheduler_type": tune.choice(["linear","cosine","polynomial"]),
    "warmup_steps": tune.uniform(100, 2000),
    "seed": tune.uniform(0,100),
    "per_device_train_batch_size": tune.choice([geneformer_batch_size])
}

In [73]:
hyperopt_search = HyperOptSearch(
    metric="eval_accuracy", mode="max")

In [74]:
# optimize hyperparameters
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    resources_per_trial={"cpu":8,"gpu":1},
    hp_space=lambda _: ray_config,
    search_alg=hyperopt_search,
    n_trials=100, # number of trials
    progress_reporter=tune.CLIReporter(max_report_frequency=600,
                                                   sort_by_metric=True,
                                                   max_progress_rows=100,
                                                   mode="max",
                                                   metric="eval_accuracy",
                                                   metric_columns=["loss", "eval_loss", "eval_accuracy","macro_f1"])
)

DeprecationWarning: Accepting a `checkpoint_dir` argument in your training function is deprecated.
Please use `ray.train.get_checkpoint()` to access your checkpoint as a
`ray.train.Checkpoint` object instead. See below for an example:

Before
------

from ray import tune

def train_fn(config, checkpoint_dir=None):
    if checkpoint_dir:
        torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

After
-----

from ray import train, tune

def train_fn(config):
    checkpoint: train.Checkpoint = train.get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

In [75]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.6701, 'learning_rate': 4.989965986394558e-05, 'epoch': 0.1}
{'eval_loss': 0.6622763276100159, 'eval_accuracy': 0.6060205094277208, 'eval_macro_f1': 0.5166303306330686, 'eval_runtime': 13.7348, 'eval_samples_per_second': 220.098, 'eval_steps_per_second': 18.348, 'epoch': 0.1}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.6612, 'learning_rate': 4.979931972789116e-05, 'epoch': 0.2}
{'eval_loss': 0.6730164289474487, 'eval_accuracy': 0.5954349983460139, 'eval_macro_f1': 0.39291935830022034, 'eval_runtime': 13.6878, 'eval_samples_per_second': 220.853, 'eval_steps_per_second': 18.41, 'epoch': 0.2}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


...

/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.0573, 'learning_rate': 3.7256802721088436e-05, 'epoch': 12.74}
{'eval_loss': 0.6257275342941284, 'eval_accuracy': 0.9030764141581211, 'eval_macro_f1': 0.9000026982389983, 'eval_runtime': 13.7713, 'eval_samples_per_second': 219.515, 'eval_steps_per_second': 18.299, 'epoch': 12.74}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.0646, 'learning_rate': 3.7156462585034013e-05, 'epoch': 12.84}


KeyboardInterrupt: 

In [76]:
predictions = trainer.predict(classifier_testdset)

In [77]:
import pickle
with open(f"{output_dir}predictions.pickle", "wb") as fp:
        pickle.dump(predictions, fp)
trainer.save_metrics("eval",predictions.metrics)
trainer.save_model(output_dir)

In [3]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats

In [5]:
isp = InSilicoPerturber(perturb_type="delete",
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier",
                        num_classes=2,
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,  #{"cell_type":["CD8-positive, alpha-beta T cell"]}
                        cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []},
                        max_ncells=1000,
                        emb_layer=0,
                        forward_batch_size=50,
                        nproc=4)

isp.perturb_data("/root/autodl-tmp/T_cell/model/231116_geneformer_DiseaseClassifier",
                 "/root/autodl-tmp/T_cell.dataset",
                 "/root/autodl-tmp/T_cell/",
                 "T_cell_perturbation")

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/20152 [00:00<?, ? examples/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


Map:   0%|          | 0/365 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/365 [00:00<?, ? examples/s]

Map:   0%|          | 0/358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/358 [00:00<?, ? examples/s]

In [6]:
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all",
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []})

In [7]:
ispstats.get_stats("/root/autodl-tmp/T_cell",
                   None,
                   "/root/autodl-tmp/T_cell/",
                   "T_cell")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12172 [00:00<?, ?it/s]

  0%|          | 0/12172 [00:00<?, ?it/s]